In [1]:
# Built-in library
import re
import json
import logging
from typing import Any, Dict, List, Optional, Union
import logging
import warnings

# Standard imports
import numpy as np
import pandas as pd
import polars as pl
from rich import print
import torch

# Visualization
import matplotlib.pyplot as plt


# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

### Create A DataFrame

In [2]:
from datetime import datetime


df: pl.DataFrame = pl.DataFrame(
    {
        "integer": [1, 2, 3, 4, 5],
        "date": [
            datetime(2022, 1, 1),
            datetime(2022, 1, 2),
            datetime(2022, 1, 3),
            datetime(2022, 1, 4),
            datetime(2022, 1, 5),
        ],
        "float": [4.0, 5.0, 6.0, 7.0, 8.0],
    }
)

print(df)

shape: (5, 3)
┌─────────┬─────────────────────┬───────┐
│ integer ┆ date                ┆ float │
│ ---     ┆ ---                 ┆ ---   │
│ i64     ┆ datetime[μs]        ┆ f64   │
╞═════════╪═════════════════════╪═══════╡
│ 1       ┆ 2022-01-01 00:00:00 ┆ 4.0   │
│ 2       ┆ 2022-01-02 00:00:00 ┆ 5.0   │
│ 3       ┆ 2022-01-03 00:00:00 ┆ 6.0   │
│ 4       ┆ 2022-01-04 00:00:00 ┆ 7.0   │
│ 5       ┆ 2022-01-05 00:00:00 ┆ 8.0   │
└─────────┴─────────────────────┴───────┘

#### Viewing Data

```text
- df.head()
- df.tail()
```

In [3]:
df.head(2)

integer,date,float
i64,datetime[μs],f64
1,2022-01-01 00:00:00,4.0
2,2022-01-02 00:00:00,5.0


In [4]:
df.tail(2)

integer,date,float
i64,datetime[μs],f64
4,2022-01-04 00:00:00,7.0
5,2022-01-05 00:00:00,8.0


#### Descriptive Stats

In [5]:
df.describe()

describe,integer,date,float
str,f64,str,f64
"""count""",5.0,"""5""",5.0
"""null_count""",0.0,"""0""",0.0
"""mean""",3.0,null,6.0
"""std""",1.581139,null,1.581139
"""min""",1.0,"""2022-01-01 00:…",4.0
"""25%""",2.0,null,5.0
"""50%""",3.0,null,6.0
"""75%""",4.0,null,7.0
"""max""",5.0,"""2022-01-05 00:…",8.0


In [6]:
df: pl.DataFrame = pl.DataFrame(
    {
        "nrs": [1, 2, 3, None, 5],
        "names": ["foo", "ham", "spam", "egg", None],
        "random": np.random.rand(5),
        "groups": ["A", "A", "B", "C", "B"],
    }
)
print(df)

shape: (5, 4)
┌──────┬───────┬──────────┬────────┐
│ nrs  ┆ names ┆ random   ┆ groups │
│ ---  ┆ ---   ┆ ---      ┆ ---    │
│ i64  ┆ str   ┆ f64      ┆ str    │
╞══════╪═══════╪══════════╪════════╡
│ 1    ┆ foo   ┆ 0.397754 ┆ A      │
│ 2    ┆ ham   ┆ 0.639284 ┆ A      │
│ 3    ┆ spam  ┆ 0.625305 ┆ B      │
│ null ┆ egg   ┆ 0.23468  ┆ C      │
│ 5    ┆ null  ┆ 0.3238   ┆ B      │
└──────┴───────┴──────────┴────────┘

In [7]:
# Select
out: pl.DataFrame = df.select(
    pl.sum("nrs"),  # sum this column
    pl.col("names").sort(),  # select and sort this column
    # select the first row and rename this column
    pl.col("names").first().alias("first name"),
    # calculate the mean, multiply by 10 and rename
    (pl.mean("nrs") * 10).alias("10xnrs"),
)
print(out)

shape: (5, 4)
┌─────┬───────┬────────────┬────────┐
│ nrs ┆ names ┆ first name ┆ 10xnrs │
│ --- ┆ ---   ┆ ---        ┆ ---    │
│ i64 ┆ str   ┆ str        ┆ f64    │
╞═════╪═══════╪════════════╪════════╡
│ 11  ┆ null  ┆ foo        ┆ 27.5   │
│ 11  ┆ egg   ┆ foo        ┆ 27.5   │
│ 11  ┆ foo   ┆ foo        ┆ 27.5   │
│ 11  ┆ ham   ┆ foo        ┆ 27.5   │
│ 11  ┆ spam  ┆ foo        ┆ 27.5   │
└─────┴───────┴────────────┴────────┘

In [8]:
df

nrs,names,random,groups
i64,str,f64,str
1,"""foo""",0.397754,"""A"""
2,"""ham""",0.639284,"""A"""
3,"""spam""",0.625305,"""B"""
null,"""egg""",0.23468,"""C"""
5,null,0.3238,"""B"""


In [9]:
# Add new column(s)
df_1: pl.DataFrame = df.with_columns(
    pl.sum("nrs").alias("nrs_sum"),
    pl.col("random").count().alias("count"),
)
print(df_1)

shape: (5, 6)
┌──────┬───────┬──────────┬────────┬─────────┬───────┐
│ nrs  ┆ names ┆ random   ┆ groups ┆ nrs_sum ┆ count │
│ ---  ┆ ---   ┆ ---      ┆ ---    ┆ ---     ┆ ---   │
│ i64  ┆ str   ┆ f64      ┆ str    ┆ i64     ┆ u32   │
╞══════╪═══════╪══════════╪════════╪═════════╪═══════╡
│ 1    ┆ foo   ┆ 0.397754 ┆ A      ┆ 11      ┆ 5     │
│ 2    ┆ ham   ┆ 0.639284 ┆ A      ┆ 11      ┆ 5     │
│ 3    ┆ spam  ┆ 0.625305 ┆ B      ┆ 11      ┆ 5     │
│ null ┆ egg   ┆ 0.23468  ┆ C      ┆ 11      ┆ 5     │
│ 5    ┆ null  ┆ 0.3238   ┆ B      ┆ 11      ┆ 5     │
└──────┴───────┴──────────┴────────┴─────────┴───────┘

In [10]:
# Filter
out: pl.DataFrame = df_1.filter(pl.col("nrs") > 2)
print(out)

shape: (2, 6)
┌─────┬───────┬──────────┬────────┬─────────┬───────┐
│ nrs ┆ names ┆ random   ┆ groups ┆ nrs_sum ┆ count │
│ --- ┆ ---   ┆ ---      ┆ ---    ┆ ---     ┆ ---   │
│ i64 ┆ str   ┆ f64      ┆ str    ┆ i64     ┆ u32   │
╞═════╪═══════╪══════════╪════════╪═════════╪═══════╡
│ 3   ┆ spam  ┆ 0.625305 ┆ B      ┆ 11      ┆ 5     │
│ 5   ┆ null  ┆ 0.3238   ┆ B      ┆ 11      ┆ 5     │
└─────┴───────┴──────────┴────────┴─────────┴───────┘

### Group By

In [11]:
print(df_1)

shape: (5, 6)
┌──────┬───────┬──────────┬────────┬─────────┬───────┐
│ nrs  ┆ names ┆ random   ┆ groups ┆ nrs_sum ┆ count │
│ ---  ┆ ---   ┆ ---      ┆ ---    ┆ ---     ┆ ---   │
│ i64  ┆ str   ┆ f64      ┆ str    ┆ i64     ┆ u32   │
╞══════╪═══════╪══════════╪════════╪═════════╪═══════╡
│ 1    ┆ foo   ┆ 0.397754 ┆ A      ┆ 11      ┆ 5     │
│ 2    ┆ ham   ┆ 0.639284 ┆ A      ┆ 11      ┆ 5     │
│ 3    ┆ spam  ┆ 0.625305 ┆ B      ┆ 11      ┆ 5     │
│ null ┆ egg   ┆ 0.23468  ┆ C      ┆ 11      ┆ 5     │
│ 5    ┆ null  ┆ 0.3238   ┆ B      ┆ 11      ┆ 5     │
└──────┴───────┴──────────┴────────┴─────────┴───────┘

In [12]:
out: pl.DataFrame = df_1.group_by("groups").agg(
    pl.sum("nrs"),  # sum nrs by groups
    pl.col("random").count().alias("count"),  # count group members
    # sum random where name != null
    pl.col("random").filter(pl.col("names").is_not_null()).sum().name.suffix("_sum"),
    pl.col("names").reverse().alias("reversed names"),
)
print(out)

shape: (3, 5)
┌────────┬─────┬───────┬────────────┬────────────────┐
│ groups ┆ nrs ┆ count ┆ random_sum ┆ reversed names │
│ ---    ┆ --- ┆ ---   ┆ ---        ┆ ---            │
│ str    ┆ i64 ┆ u32   ┆ f64        ┆ list[str]      │
╞════════╪═════╪═══════╪════════════╪════════════════╡
│ B      ┆ 8   ┆ 2     ┆ 0.625305   ┆ [null, "spam"] │
│ A      ┆ 3   ┆ 2     ┆ 1.037038   ┆ ["ham", "foo"] │
│ C      ┆ 0   ┆ 1     ┆ 0.23468    ┆ ["egg"]        │
└────────┴─────┴───────┴────────────┴────────────────┘

### Expressions

```text
- Polars has a powerful concept called expressions that is central to its very fast performance.
- Expressions are at the core of many data science operations:
  * taking a sample of rows from a column
  * multiplying values in a column
  * extracting a column of years from dates
  * convert a column of strings to lowercase

- However, expressions are also used within other operations:
  * taking the mean of a group in a group_by operation
  * calculating the size of groups in a group_by operation
  * taking the sum horizontally across columns
  * Polars performs these core data transformations very quickly by:

- Polars expressions are a mapping from a series to a series (or mathematically Fn(Series) -> Series). 
- As expressions have a Series as an input and a Series as an output then it is straightforward to do a sequence of expressions (similar to method chaining in Pandas).
```

In [13]:
# Select column "foo"
# Then sort the column (not in reversed order)
# Then take the first two values of the sorted output
pl.col("foo").sort().head(2)

In [14]:
df_1

nrs,names,random,groups,nrs_sum,count
i64,str,f64,str,i64,u32
1,"""foo""",0.397754,"""A""",11,5
2,"""ham""",0.639284,"""A""",11,5
3,"""spam""",0.625305,"""B""",11,5
null,"""egg""",0.23468,"""C""",11,5
5,null,0.3238,"""B""",11,5


In [15]:
df

nrs,names,random,groups
i64,str,f64,str
1,"""foo""",0.397754,"""A"""
2,"""ham""",0.639284,"""A"""
3,"""spam""",0.625305,"""B"""
null,"""egg""",0.23468,"""C"""
5,null,0.3238,"""B"""


### [Lazy Vs Eager API](https://pola-rs.github.io/polars/user-guide/concepts/lazy-vs-eager/)

In [16]:
fp: str = "../../data/AirlineTweets.csv"
df: pl.DataFrame = pl.read_csv(fp)
pl.Config.set_tbl_column_data_type_inline(True)
N: int = 8
with pl.Config(tbl_cols=N):  # display N columns
    print(df.head())

shape: (5, 15)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ tweet_id  ┆ airline_s ┆ airline_s ┆ negativer ┆ … ┆ tweet_coo ┆ tweet_cre ┆ tweet_loc ┆ user_tim │
│ (i64)     ┆ entiment  ┆ entiment_ ┆ eason     ┆   ┆ rd (str)  ┆ ated      ┆ ation     ┆ ezone    │
│           ┆ (str)     ┆ confidenc ┆ (str)     ┆   ┆           ┆ (str)     ┆ (str)     ┆ (str)    │
│           ┆           ┆ e (f64)   ┆           ┆   ┆           ┆           ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 570306133 ┆ neutral   ┆ 1.0       ┆ null      ┆ … ┆ null      ┆ 2015-02-2 ┆ null      ┆ Eastern  │
│ 677760513 ┆           ┆           ┆           ┆   ┆           ┆ 4         ┆           ┆ Time (US │
│           ┆           ┆           ┆           ┆   ┆           ┆ 11:35:52  ┆           ┆ &        │
│           ┆           ┆           ┆           ┆   ┆           ┆ -0800     ┆           ┆ Canada)  │
│ 570301130 ┆ positive  ┆ 0.3486    ┆ null      ┆ … ┆ null      ┆ 2015-02-2 ┆ null      ┆ Pacific  │
│ 888122368 ┆           ┆           ┆           ┆   ┆           ┆ 4         ┆           ┆ Time (US │
│           ┆           ┆           ┆           ┆   ┆           ┆ 11:15:59  ┆           ┆ &        │
│           ┆           ┆           ┆           ┆   ┆           ┆ -0800     ┆           ┆ Canada)  │
│ 570301083 ┆ neutral   ┆ 0.6837    ┆ null      ┆ … ┆ null      ┆ 2015-02-2 ┆ Lets Play ┆ Central  │
│ 672813571 ┆           ┆           ┆           ┆   ┆           ┆ 4         ┆           ┆ Time (US │
│           ┆           ┆           ┆           ┆   ┆           ┆ 11:15:48  ┆           ┆ &        │
│           ┆           ┆           ┆           ┆   ┆           ┆ -0800     ┆           ┆ Canada)  │
│ 570301031 ┆ negative  ┆ 1.0       ┆ Bad       ┆ … ┆ null      ┆ 2015-02-2 ┆ null      ┆ Pacific  │
│ 407624196 ┆           ┆           ┆ Flight    ┆   ┆           ┆ 4         ┆           ┆ Time (US │
│           ┆           ┆           ┆           ┆   ┆           ┆ 11:15:36  ┆           ┆ &        │
│           ┆           ┆           ┆           ┆   ┆           ┆ -0800     ┆           ┆ Canada)  │
│ 570300817 ┆ negative  ┆ 1.0       ┆ Can't     ┆ … ┆ null      ┆ 2015-02-2 ┆ null      ┆ Pacific  │
│ 074462722 ┆           ┆           ┆ Tell      ┆   ┆           ┆ 4         ┆           ┆ Time (US │
│           ┆           ┆           ┆           ┆   ┆           ┆ 11:14:45  ┆           ┆ &        │
│           ┆           ┆           ┆           ┆   ┆           ┆ -0800     ┆           ┆ Canada)  │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴───────────┴──────────┘

In [17]:
df.describe()

describe,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
str,f64,str,f64,str,f64,str,str,str,str,f64,str,str,str,str,str
"""count""",14640.0,"""14640""",14640.0,"""14640""",14640.0,"""14640""","""14640""","""14640""","""14640""",14640.0,"""14640""","""14640""","""14640""","""14640""","""14640"""
"""null_count""",0.0,"""0""",0.0,"""5462""",4118.0,"""0""","""14600""","""0""","""14608""",0.0,"""0""","""13621""","""0""","""4733""","""4820"""
"""mean""",5.6922e17,null,0.900169,null,0.638298,null,null,null,null,0.08265,null,null,null,null,null
"""std""",7.7911e14,null,0.16283,null,0.33044,null,null,null,null,0.745778,null,null,null,null,null
"""min""",5.6759e17,"""negative""",0.335,"""Bad Flight""",0.0,"""American""","""negative""","""0504Traveller""","""Bad Flight""",0.0,"""""LOL you guys …","""[-33.87144962,…","""2015-02-16 23:…",""" || san anton…","""Abu Dhabi"""
"""25%""",5.6856e17,null,0.6923,null,0.3606,null,null,null,null,0.0,null,null,null,null,null
"""50%""",5.6948e17,null,1.0,null,0.6706,null,null,null,null,0.0,null,null,null,null,null
"""75%""",5.6989e17,null,1.0,null,1.0,null,null,null,null,0.0,null,null,null,null,null
"""max""",5.7031e17,"""positive""",1.0,"""longlines""",1.0,"""Virgin America…","""positive""","""zupshawrl""","""Lost Luggage D…",44.0,"""😳 LOLOLOLOLOL …","""[59.38247253, …","""2015-02-24 11:…","""명동서식 37.56638,…","""West Central A…"


In [18]:
VALUE: float = 0.567
df_small: pl.DataFrame = df.filter(pl.col("airline_sentiment_confidence") > VALUE)
df_agg: pl.DataFrame = df_small.group_by("negativereason").agg(
    pl.col("retweet_count").mean()
)
print(df_agg)

shape: (11, 2)
┌─────────────────────────┬─────────────────────┐
│ negativereason (str)    ┆ retweet_count (f64) │
╞═════════════════════════╪═════════════════════╡
│ Customer Service Issue  ┆ 0.076128            │
│ Can't Tell              ┆ 0.187983            │
│ Bad Flight              ┆ 0.076256            │
│ Late Flight             ┆ 0.093656            │
│ …                       ┆ …                   │
│ null                    ┆ 0.065797            │
│ longlines               ┆ 0.050562            │
│ Flight Booking Problems ┆ 0.080614            │
│ Cancelled Flight        ┆ 0.07346             │
└─────────────────────────┴─────────────────────┘

### Lazy API

```text
- These will significantly lower the load on memory & CPU thus allowing you to fit bigger datasets in memory and process faster. 
- Once the query is defined you call collect to inform Polars that you want to execute it.
```

In [19]:
q = (
    pl.scan_csv(fp)
    .filter(pl.col("airline_sentiment_confidence") > VALUE)
    .group_by("negativereason")
    .agg(pl.col("retweet_count").mean())
)

df = q.collect()
df

negativereason,retweet_count
str,f64
"""Can't Tell""",0.187983
"""Flight Attenda…",0.090336
"""Customer Servi…",0.076128
"""Flight Booking…",0.080614
"""Lost Luggage""",0.073407
"""Cancelled Flig…",0.07346
null,0.065797
"""longlines""",0.050562
"""Damaged Luggag…",0.068493


#### Streaming API

```text 
- One additional benefit of the lazy API is that it allows queries to be executed in a streaming manner. 
- Instead of processing the data all-at-once Polars can execute the query in batches allowing you to process datasets that are larger-than-memory.
- To tell Polars we want to execute a query in streaming mode we pass the streaming=True argument to collect
```

In [20]:
q = (
    pl.scan_csv(fp)
    .filter(pl.col("airline_sentiment_confidence") > VALUE)
    .group_by("negativereason")
    .agg(pl.col("retweet_count").mean())
)

df = q.collect(streaming=True)
df

negativereason,retweet_count
str,f64
"""Cancelled Flig…",0.07346
"""Flight Attenda…",0.090336
"""Bad Flight""",0.076256
"""Late Flight""",0.093656
"""Damaged Luggag…",0.068493
"""Lost Luggage""",0.073407
"""Customer Servi…",0.076128
null,0.065797
"""Can't Tell""",0.187983


### Basic Operator

In [21]:
df: pl.DataFrame = pl.DataFrame(
    {
        "nrs": [1, 2, 3, None, 5],
        "names": ["foo", "ham", "spam", "egg", None],
        "random": np.random.rand(5),
        "groups": ["A", "A", "B", "C", "B"],
    }
)
print(df)

shape: (5, 4)
┌───────────┬─────────────┬──────────────┬──────────────┐
│ nrs (i64) ┆ names (str) ┆ random (f64) ┆ groups (str) │
╞═══════════╪═════════════╪══════════════╪══════════════╡
│ 1         ┆ foo         ┆ 0.468765     ┆ A            │
│ 2         ┆ ham         ┆ 0.460675     ┆ A            │
│ 3         ┆ spam        ┆ 0.104761     ┆ B            │
│ null      ┆ egg         ┆ 0.304853     ┆ C            │
│ 5         ┆ null        ┆ 0.72567      ┆ B            │
└───────────┴─────────────┴──────────────┴──────────────┘

In [22]:
df_logical: pl.DataFrame = df.select(
    (pl.col("nrs") > 1).alias("nrs > 1"),
    (pl.col("random") <= 0.5).alias("random <= .5"),
    (pl.col("nrs") != 1).alias("nrs != 1"),
    (pl.col("nrs") == 1).alias("nrs == 1"),
    ((pl.col("random") <= 0.5) & (pl.col("nrs") > 1)).alias("and_expr"),  # and
    ((pl.col("random") <= 0.5) | (pl.col("nrs") > 1)).alias("or_expr"),  # or
)
print(df_logical)

shape: (5, 6)
┌─────────┬──────────────┬──────────┬─────────────────┬─────────────────┬────────────────┐
│ nrs > 1 ┆ random <= .5 ┆ nrs != 1 ┆ nrs == 1 (bool) ┆ and_expr (bool) ┆ or_expr (bool) │
│ (bool)  ┆ (bool)       ┆ (bool)   ┆                 ┆                 ┆                │
╞═════════╪══════════════╪══════════╪═════════════════╪═════════════════╪════════════════╡
│ false   ┆ true         ┆ false    ┆ true            ┆ false           ┆ true           │
│ true    ┆ true         ┆ true     ┆ false           ┆ true            ┆ true           │
│ true    ┆ true         ┆ true     ┆ false           ┆ true            ┆ true           │
│ null    ┆ true         ┆ null     ┆ null            ┆ null            ┆ true           │
│ true    ┆ false        ┆ true     ┆ false           ┆ false           ┆ true           │
└─────────┴──────────────┴──────────┴─────────────────┴─────────────────┴────────────────┘

In [23]:
from datetime import date, datetime


df: pl.DataFrame = pl.DataFrame(
    {
        "id": [9, 4, 2],
        "place": ["Mars", "Earth", "Saturn"],
        "date": pl.date_range(date(2022, 1, 1), date(2022, 1, 3), "1d", eager=True),
        "sales": [33.4, 2142134.1, 44.7],
        "has_people": [False, True, False],
        "logged_at": pl.datetime_range(
            datetime(2022, 12, 1), datetime(2022, 12, 1, 0, 0, 2), "1s", eager=True
        ),
    }
    # Add a column at index 0 that counts the rows.
).with_row_count("rn")
print(df)

shape: (3, 7)
┌──────────┬──────────┬─────────────┬─────────────┬─────────────┬────────────┬─────────────────────┐
│ rn (u32) ┆ id (i64) ┆ place (str) ┆ date (date) ┆ sales (f64) ┆ has_people ┆ logged_at           │
│          ┆          ┆             ┆             ┆             ┆ (bool)     ┆ (datetime[μs])      │
╞══════════╪══════════╪═════════════╪═════════════╪═════════════╪════════════╪═════════════════════╡
│ 0        ┆ 9        ┆ Mars        ┆ 2022-01-01  ┆ 33.4        ┆ false      ┆ 2022-12-01 00:00:00 │
│ 1        ┆ 4        ┆ Earth       ┆ 2022-01-02  ┆ 2142134.1   ┆ true       ┆ 2022-12-01 00:00:01 │
│ 2        ┆ 2        ┆ Saturn      ┆ 2022-01-03  ┆ 44.7        ┆ false      ┆ 2022-12-01 00:00:02 │
└──────────┴──────────┴─────────────┴─────────────┴─────────────┴────────────┴─────────────────────┘

In [24]:
out = df.select(pl.col("*"))

# Is equivalent to
out = df.select(pl.all())
print(out)

shape: (3, 7)
┌──────────┬──────────┬─────────────┬─────────────┬─────────────┬────────────┬─────────────────────┐
│ rn (u32) ┆ id (i64) ┆ place (str) ┆ date (date) ┆ sales (f64) ┆ has_people ┆ logged_at           │
│          ┆          ┆             ┆             ┆             ┆ (bool)     ┆ (datetime[μs])      │
╞══════════╪══════════╪═════════════╪═════════════╪═════════════╪════════════╪═════════════════════╡
│ 0        ┆ 9        ┆ Mars        ┆ 2022-01-01  ┆ 33.4        ┆ false      ┆ 2022-12-01 00:00:00 │
│ 1        ┆ 4        ┆ Earth       ┆ 2022-01-02  ┆ 2142134.1   ┆ true       ┆ 2022-12-01 00:00:01 │
│ 2        ┆ 2        ┆ Saturn      ┆ 2022-01-03  ┆ 44.7        ┆ false      ┆ 2022-12-01 00:00:02 │
└──────────┴──────────┴─────────────┴─────────────┴─────────────┴────────────┴─────────────────────┘

In [25]:
# exclude

out = df.select(pl.col("*").exclude("logged_at", "rn"))
print(out)

shape: (3, 5)
┌──────────┬─────────────┬─────────────┬─────────────┬───────────────────┐
│ id (i64) ┆ place (str) ┆ date (date) ┆ sales (f64) ┆ has_people (bool) │
╞══════════╪═════════════╪═════════════╪═════════════╪═══════════════════╡
│ 9        ┆ Mars        ┆ 2022-01-01  ┆ 33.4        ┆ false             │
│ 4        ┆ Earth       ┆ 2022-01-02  ┆ 2142134.1   ┆ true              │
│ 2        ┆ Saturn      ┆ 2022-01-03  ┆ 44.7        ┆ false             │
└──────────┴─────────────┴─────────────┴─────────────┴───────────────────┘

### [Select By Data Type](https://pola-rs.github.io/polars/user-guide/expressions/column-selections/#by-regular-expressions)

In [26]:
out = df.select(pl.col(pl.Int64, pl.UInt32, pl.Boolean).n_unique())
print(out)

shape: (1, 3)
┌──────────┬──────────┬──────────────────┐
│ rn (u32) ┆ id (u32) ┆ has_people (u32) │
╞══════════╪══════════╪══════════════════╡
│ 3        ┆ 3        ┆ 2                │
└──────────┴──────────┴──────────────────┘

In [27]:
#  selectors

import polars.selectors as cs


out = df.select(cs.integer(), cs.string())
print(out)

shape: (3, 3)
┌──────────┬──────────┬─────────────┐
│ rn (u32) ┆ id (i64) ┆ place (str) │
╞══════════╪══════════╪═════════════╡
│ 0        ┆ 9        ┆ Mars        │
│ 1        ┆ 4        ┆ Earth       │
│ 2        ┆ 2        ┆ Saturn      │
└──────────┴──────────┴─────────────┘

In [28]:
df_1

nrs,names,random,groups,nrs_sum,count
i64,str,f64,str,i64,u32
1,"""foo""",0.397754,"""A""",11,5
2,"""ham""",0.639284,"""A""",11,5
3,"""spam""",0.625305,"""B""",11,5
null,"""egg""",0.23468,"""C""",11,5
5,null,0.3238,"""B""",11,5


In [29]:
df_alias = df_1.select(
    # Count of unique values
    pl.col("names").n_unique().alias("unique"),
    # Approximate count of unique values
    pl.approx_n_unique("names").alias("unique_approx"),
)
print(df_alias)

shape: (1, 2)
┌──────────────┬─────────────────────┐
│ unique (u32) ┆ unique_approx (u32) │
╞══════════════╪═════════════════════╡
│ 5            ┆ 5                   │
└──────────────┴─────────────────────┘

### Conditionals

In [30]:
df_1

nrs,names,random,groups,nrs_sum,count
i64,str,f64,str,i64,u32
1,"""foo""",0.397754,"""A""",11,5
2,"""ham""",0.639284,"""A""",11,5
3,"""spam""",0.625305,"""B""",11,5
null,"""egg""",0.23468,"""C""",11,5
5,null,0.3238,"""B""",11,5


In [31]:
# when

df_conditional = df_1.select(
    pl.col("nrs"),
    pl.when(pl.col("nrs") > 2)
    .then(pl.lit(True))
    .otherwise(pl.lit(False))
    .alias("conditional"),
)
print(df_conditional)

shape: (5, 2)
┌───────────┬────────────────────┐
│ nrs (i64) ┆ conditional (bool) │
╞═══════════╪════════════════════╡
│ 1         ┆ false              │
│ 2         ┆ false              │
│ 3         ┆ true               │
│ null      ┆ false              │
│ 5         ┆ true               │
└───────────┴────────────────────┘

### Strings

In [32]:
# str.len_bytes str.len_chars

df: pl.DataFrame = pl.DataFrame({"animal": ["Crab", "cat and dog", "rab$bit", None]})
print(df)

shape: (4, 1)
┌──────────────┐
│ animal (str) │
╞══════════════╡
│ Crab         │
│ cat and dog  │
│ rab$bit      │
│ null         │
└──────────────┘

In [33]:
# Count the number of characters
out: pl.DataFrame = df.select(
    pl.col("animal").str.len_bytes().alias("byte_count"),
    pl.col("animal").str.len_chars().alias("letter_count"),
)
print(out)

shape: (4, 2)
┌──────────────────┬────────────────────┐
│ byte_count (u32) ┆ letter_count (u32) │
╞══════════════════╪════════════════════╡
│ 4                ┆ 4                  │
│ 11               ┆ 11                 │
│ 7                ┆ 7                  │
│ null             ┆ null               │
└──────────────────┴────────────────────┘

In [34]:
#  str.contains str.starts_with str.ends_with

out: pl.DataFrame = df.select(
    pl.col("animal"),
    pl.col("animal").str.contains("cat|bit").alias("regex"),
    pl.col("animal").str.contains("rab$", literal=True).alias("literal"),
    pl.col("animal").str.starts_with("rab").alias("starts_with"),
    pl.col("animal").str.ends_with("dog").alias("ends_with"),
)
print(out)

shape: (4, 5)
┌──────────────┬──────────────┬────────────────┬────────────────────┬──────────────────┐
│ animal (str) ┆ regex (bool) ┆ literal (bool) ┆ starts_with (bool) ┆ ends_with (bool) │
╞══════════════╪══════════════╪════════════════╪════════════════════╪══════════════════╡
│ Crab         ┆ false        ┆ false          ┆ false              ┆ false            │
│ cat and dog  ┆ true         ┆ false          ┆ false              ┆ true             │
│ rab$bit      ┆ true         ┆ true           ┆ true               ┆ false            │
│ null         ┆ null         ┆ null           ┆ null               ┆ null             │
└──────────────┴──────────────┴────────────────┴────────────────────┴──────────────────┘

In [35]:
# str.extract

df: pl.DataFrame = pl.DataFrame(
    {
        "a": [
            "http://vote.com/ballon_dor?candidate=messi&ref=polars",
            "http://vote.com/ballon_dor?candidat=jorginho&ref=polars",
            "http://vote.com/ballon_dor?candidate=ronaldo&ref=polars",
        ]
    }
)
print(df)

shape: (3, 1)
┌───────────────────────────────────┐
│ a (str)                           │
╞═══════════════════════════════════╡
│ http://vote.com/ballon_dor?candi… │
│ http://vote.com/ballon_dor?candi… │
│ http://vote.com/ballon_dor?candi… │
└───────────────────────────────────┘

In [36]:
out: pl.DataFrame = df.select(
    pl.col("a").str.extract(r"candidate=(\w+)", group_index=1),
)
print(out)

shape: (3, 1)
┌─────────┐
│ a (str) │
╞═════════╡
│ messi   │
│ null    │
│ ronaldo │
└─────────┘

In [37]:
out: pl.DataFrame = df.select(
    pl.col("a").str.extract(r"(candidate=\w+)", group_index=1),
)
print(out)

shape: (3, 1)
┌───────────────────┐
│ a (str)           │
╞═══════════════════╡
│ candidate=messi   │
│ null              │
│ candidate=ronaldo │
└───────────────────┘

In [38]:
# str.extract_all

df: pl.DataFrame = pl.DataFrame({"foo": ["123 bla 45 asd", "xyz 678 910t"]})
print(df)

shape: (2, 1)
┌────────────────┐
│ foo (str)      │
╞════════════════╡
│ 123 bla 45 asd │
│ xyz 678 910t   │
└────────────────┘

In [39]:
out = df.select(
    pl.col("foo").str.extract_all(r"(\d+)").alias("extracted_nrs"),
)
print(out)

shape: (2, 1)
┌───────────────────────────┐
│ extracted_nrs (list[str]) │
╞═══════════════════════════╡
│ ["123", "45"]             │
│ ["678", "910"]            │
└───────────────────────────┘

In [40]:
#  str.replace str.replace_all

df: pl.DataFrame = pl.DataFrame({"id": [1, 2], "text": ["123abc", "abc456"]})
print(df)

shape: (2, 2)
┌──────────┬────────────┐
│ id (i64) ┆ text (str) │
╞══════════╪════════════╡
│ 1        ┆ 123abc     │
│ 2        ┆ abc456     │
└──────────┴────────────┘

In [41]:
# Add a new column
out: pl.DataFrame = df.with_columns(
    pl.col("text").str.replace(r"abc\b", "ABC"),
    pl.col("text").str.replace_all("a", "-", literal=True).alias("text_replace_all"),
)
print(out)

shape: (2, 3)
┌──────────┬────────────┬────────────────────────┐
│ id (i64) ┆ text (str) ┆ text_replace_all (str) │
╞══════════╪════════════╪════════════════════════╡
│ 1        ┆ 123ABC     ┆ 123-bc                 │
│ 2        ┆ abc456     ┆ -bc456                 │
└──────────┴────────────┴────────────────────────┘

<br>

### Aggregations

In [42]:
url: str = "https://theunitedstates.io/congress-legislators/legislators-historical.csv"

dtypes = {
    "first_name": pl.Categorical,
    "gender": pl.Categorical,
    "type": pl.Categorical,
    "state": pl.Categorical,
    "party": pl.Categorical,
}

dataset = (
    pl.read_csv(url, dtypes=dtypes)
    # Modifiy the column and convert to date
    .with_columns(pl.col("birthday").str.to_date(strict=False))
)

print(dataset.head())

shape: (5, 36)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ last_name ┆ first_nam ┆ middle_na ┆ suffix    ┆ … ┆ ballotped ┆ washingto ┆ icpsr_id  ┆ wikipedi │
│ (str)     ┆ e (cat)   ┆ me (str)  ┆ (str)     ┆   ┆ ia_id     ┆ n_post_id ┆ (i64)     ┆ a_id     │
│           ┆           ┆           ┆           ┆   ┆ (str)     ┆ (str)     ┆           ┆ (str)    │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ Bassett   ┆ Richard   ┆ null      ┆ null      ┆ … ┆ null      ┆ null      ┆ 507       ┆ Richard  │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ Bassett  │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ (Delawar │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ e        │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ politi…  │
│ Bland     ┆ Theodoric ┆ null      ┆ null      ┆ … ┆ null      ┆ null      ┆ 786       ┆ Theodori │
│           ┆ k         ┆           ┆           ┆   ┆           ┆           ┆           ┆ ck Bland │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ (congres │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ sman)    │
│ Burke     ┆ Aedanus   ┆ null      ┆ null      ┆ … ┆ null      ┆ null      ┆ 1260      ┆ Aedanus  │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ Burke    │
│ Carroll   ┆ Daniel    ┆ null      ┆ null      ┆ … ┆ null      ┆ null      ┆ 1538      ┆ Daniel   │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ Carroll  │
│ Clymer    ┆ George    ┆ null      ┆ null      ┆ … ┆ null      ┆ null      ┆ 1859      ┆ George   │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ Clymer   │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴───────────┴──────────┘

In [43]:
dataset.describe()

describe,last_name,first_name,middle_name,suffix,nickname,full_name,birthday,gender,type,state,district,senate_class,party,url,address,phone,contact_form,rss_url,twitter,twitter_id,facebook,youtube,youtube_id,mastodon,bioguide_id,thomas_id,opensecrets_id,lis_id,fec_ids,cspan_id,govtrack_id,votesmart_id,ballotpedia_id,washington_post_id,icpsr_id,wikipedia_id
str,str,str,str,str,str,str,str,str,str,str,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,str,str,str,f64,str
"""count""","""12139""","""12139""","""12139""","""12139""","""12139""","""12139""","""12139""","""12139""","""12139""","""12139""",12139.0,12139.0,"""12139""","""12139""","""12139""","""12139""","""12139""","""12139""","""12139""","""12139""","""12139""","""12139""","""12139""","""12139""","""12139""","""12139""","""12139""","""12139""","""12139""","""12139""",12139.0,"""12139""","""12139""","""12139""",12139.0,"""12139"""
"""null_count""","""0""","""0""","""3707""","""11711""","""11902""","""11630""","""542""","""0""","""0""","""0""",1849.0,10290.0,"""232""","""11328""","""11622""","""11626""","""11898""","""11844""","""12139""","""12139""","""12139""","""12139""","""12139""","""12139""","""0""","""10228""","""11307""","""11928""","""11433""","""11597""",0.0,"""11413""","""11758""","""12139""",222.0,"""0"""
"""mean""",null,null,null,null,null,null,null,null,null,null,8.164723,2.017847,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,405524.539748,null,null,null,7442.901821,null
"""std""",null,null,null,null,null,null,null,null,null,null,8.215047,0.806632,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,10157.52946,null,null,null,6818.010195,null
"""min""","""Aandahl""",null,"""(Jim) Guy""","""II""","""Abe""","""A. Donald McEa…","""1721-04-19""",null,null,null,-1.0,1.0,null,"""http://ackerma…","""100 Cannon HOB…","""202-224-2043""","""http://akin.ho…","""feed://olson.h…",null,null,null,null,null,null,"""A000001""","""00001""","""N00000010""","""S009""","""H0AL02087""","""1000221""",300001.0,"""0""","""Aaron Schock""",null,1.0,"""A. C. Schiffle…"
"""25%""",null,null,null,null,null,null,null,null,null,null,2.0,1.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,403135.0,null,null,null,3111.0,null
"""50%""",null,null,null,null,null,null,null,null,null,null,6.0,2.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,406257.0,null,null,null,6242.0,null
"""75%""",null,null,null,null,null,null,null,null,null,null,11.0,3.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,409380.0,null,null,null,9362.0,null
"""max""","""du Pont""",null,"""de Veaux""","""Sr.""","""of Ohio""","""Yvette Herrell…","""1995-08-01""",null,null,null,53.0,3.0,null,"""https://zeldin…","""G12 Dirksen Se…","""202-225-9901""","""https://www.to…","""https://hanabu…",null,null,null,null,null,null,"""Z000017""","""02293""","""N99999896""","""S405""","""S8WI00026""","""96047""",456872.0,"""9794""","""Yvette Herrell…",null,99342.0,"""Zephaniah Swif…"


In [56]:
print(dataset.select(pl.col("gender").value_counts(sort=True)))

shape: (2, 1)
┌────────────────────┐
│ gender (struct[2]) │
╞════════════════════╡
│ {"M",11868}        │
│ {"F",271}          │
└────────────────────┘

In [44]:
dataset.group_by(pl.col("gender")).agg(pl.count())

gender,count
cat,u32
"""M""",11868
"""F""",271


In [45]:
q: pl.LazyFrame = (
    dataset.lazy()
    .group_by("first_name")
    .agg(
        pl.count(),
        pl.col("gender"),
        pl.first("last_name"),
    )
    .sort("count", descending=True)
    .limit(5)
)

df = q.collect()
print(df)

shape: (5, 4)
┌──────────────────┬─────────────┬────────────────────┬─────────────────┐
│ first_name (cat) ┆ count (u32) ┆ gender (list[cat]) ┆ last_name (str) │
╞══════════════════╪═════════════╪════════════════════╪═════════════════╡
│ John             ┆ 1256        ┆ ["M", "M", … "M"]  ┆ Walker          │
│ William          ┆ 1022        ┆ ["M", "M", … "M"]  ┆ Few             │
│ James            ┆ 714         ┆ ["M", "M", … "M"]  ┆ Armstrong       │
│ Thomas           ┆ 454         ┆ ["M", "M", … "M"]  ┆ Tucker          │
│ Charles          ┆ 439         ┆ ["M", "M", … "M"]  ┆ Carroll         │
└──────────────────┴─────────────┴────────────────────┴─────────────────┘

In [46]:
type(q)

polars.lazyframe.frame.LazyFrame

In [47]:
q: pl.LazyFrame = (
    dataset.lazy()
    .group_by("state")
    .agg(
        (pl.col("party") == "Anti-Administration").sum().alias("anti"),
        (pl.col("party") == "Pro-Administration").sum().alias("pro"),
    )
    .sort("pro", descending=True)
    .limit(5)
)

df: pl.DataFrame = q.collect()
print(df)

shape: (5, 3)
┌─────────────┬────────────┬───────────┐
│ state (cat) ┆ anti (u32) ┆ pro (u32) │
╞═════════════╪════════════╪═══════════╡
│ CT          ┆ 0          ┆ 3         │
│ NJ          ┆ 0          ┆ 3         │
│ NC          ┆ 1          ┆ 2         │
│ SC          ┆ 0          ┆ 1         │
│ PA          ┆ 1          ┆ 1         │
└─────────────┴────────────┴───────────┘

In [48]:
# OR
q: pl.LazyFrame = (
    dataset.lazy()
    .group_by(["state", "party"])
    .agg(pl.count("party").alias("count"))
    .filter(
        (pl.col("party") == "Anti-Administration")
        | (pl.col("party") == "Pro-Administration")
    )
    .sort(by="count", descending=True)
    .limit(5)
)

df: pl.DataFrame = q.collect()

df

state,party,count
cat,cat,u32
"""CT""","""Pro-Administra…",3
"""VA""","""Anti-Administr…",3
"""NJ""","""Pro-Administra…",3
"""NC""","""Pro-Administra…",2
"""VA""","""Pro-Administra…",1


In [49]:
def compute_age() -> pl.Expr:
    return date(2023, 1, 1).year - pl.col("birthday").dt.year()


def avg_birthday(gender: str) -> pl.Expr:
    return (
        compute_age()
        .filter(pl.col("gender") == gender)
        .mean()
        .alias(f"avg {gender!r} birthday")
        .round(2)
    )


q: pl.LazyFrame = (
    dataset.lazy()
    .group_by("state")
    .agg(
        avg_birthday("M"),
        avg_birthday("F"),
        (pl.col("gender") == "M").sum().alias("# male"),
        (pl.col("gender") == "F").sum().alias("# female"),
    )
    .limit(5)
)

df: pl.DataFrame = q.collect()
print(df)

shape: (5, 5)
┌─────────────┬────────────────────────┬────────────────────────┬──────────────┬────────────────┐
│ state (cat) ┆ avg 'M' birthday (f64) ┆ avg 'F' birthday (f64) ┆ # male (u32) ┆ # female (u32) │
╞═════════════╪════════════════════════╪════════════════════════╪══════════════╪════════════════╡
│ CA          ┆ 130.29                 ┆ 82.52                  ┆ 343          ┆ 31             │
│ WA          ┆ 133.44                 ┆ 86.17                  ┆ 91           ┆ 6              │
│ MT          ┆ 137.92                 ┆ 143.0                  ┆ 51           ┆ 1              │
│ NY          ┆ 187.44                 ┆ 96.09                  ┆ 1456         ┆ 22             │
│ MO          ┆ 165.74                 ┆ 83.63                  ┆ 329          ┆ 8              │
└─────────────┴────────────────────────┴────────────────────────┴──────────────┴────────────────┘

In [50]:
def get_person() -> pl.Expr:
    return pl.col("first_name") + pl.lit(" ") + pl.col("last_name")


q: pl.LazyFrame = (
    dataset.lazy()
    .sort("birthday", descending=True) # 2000, 1999, 1998, etc
    .group_by("state")
    .agg(
        get_person().first().alias("youngest"),
        get_person().last().alias("oldest"),
        get_person().sort().first().alias("alphabetical_first"),
    )
    .limit(5)
)

df: pl.DataFrame = q.collect()
print(df)

shape: (5, 4)
┌─────────────┬───────────────────────┬───────────────────────┬──────────────────────────┐
│ state (cat) ┆ youngest (str)        ┆ oldest (str)          ┆ alphabetical_first (str) │
╞═════════════╪═══════════════════════╪═══════════════════════╪══════════════════════════╡
│ CA          ┆ Katie Hill            ┆ Edward Gilbert        ┆ Aaron Sargent            │
│ MT          ┆ Greg Gianforte        ┆ James Cavanaugh       ┆ Albert Campbell          │
│ WA          ┆ Jaime Herrera Beutler ┆ Columbia Lancaster    ┆ Addison Foster           │
│ NY          ┆ Mondaire Jones        ┆ Cornelius Schoonmaker ┆ A. Foster                │
│ MO          ┆ Vicky Hartzler        ┆ Spencer Pettis        ┆ Abram Comingo            │
└─────────────┴───────────────────────┴───────────────────────┴──────────────────────────┘

last_name,first_name,middle_name,suffix,nickname,full_name,birthday,gender,type,state,district,senate_class,party,url,address,phone,contact_form,rss_url,twitter,twitter_id,facebook,youtube,youtube_id,mastodon,bioguide_id,thomas_id,opensecrets_id,lis_id,fec_ids,cspan_id,govtrack_id,votesmart_id,ballotpedia_id,washington_post_id,icpsr_id,wikipedia_id
str,cat,str,str,str,str,date,cat,cat,cat,i64,i64,cat,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,i64,str
"""Hill""","""Katie""",null,null,null,"""Katie Hill""",1987-08-25,"""F""","""rep""","""CA""",25,null,"""Democrat""","""https://katieh…","""1130 Longworth…","""202-225-1956""",null,null,null,null,null,null,null,null,"""H001087""",null,"""N00040644""",null,"""H8CA25074""",null,412756,"""179354""","""Katie Hill""",null,21932,"""Katie Hill (po…"
